# PART1

## 1. ESTUDIO PREVIO DE LOS ARCHIVOS
En este apartado se realizarán comprobaciones para verificar que la información sobre los campos dada en los PDFs es correcta.
Por ejemplo, el archivo 'actividadeseconomicas' puede tener campos repetidos, es decir, que ya se encuentran en el archivo 'locales'. Por lo que sería bastante eficiente eliminar esos campos duplicados, los cuales pueden ser unidos por su clave foranea ID_LOCAL en el futuro.
Esto proporcionaría velocidad de consulta , ahorro de espacio y de computo a futuro.

In [53]:
#------------Dependencies-----------#
import pandas as pd
import pymongo as mongo

In [54]:
#Path to files
path_actividad = "data_files/actividadeconomica202312.csv"
path_licencias = "data_files/licencias202312.csv"
path_locales = "data_files/locales202312.csv"
path_terrazas = "data_files/terrazas202312.csv"


**Dataframe creation 'Actividad Economica'**

In [55]:
df_actividad = pd.read_csv(
    path_actividad,
    sep = ";",
    skip_blank_lines = True,
    on_bad_lines = 'skip',
    engine = 'python',
    encoding = 'utf-8'
)

**Dataframe creation 'Licencias'**

In [56]:
df_licencias = pd.read_csv(path_licencias,sep = ";",skip_blank_lines = True,on_bad_lines = 'skip',engine = 'python',  encoding = 'utf-8')

**Dataframe creation 'Locales'**

In [57]:
df_locales = pd.read_csv(path_locales,sep = ";",skip_blank_lines = True,on_bad_lines = 'skip',engine = 'python',  encoding = 'utf-8')

**Dataframe creation 'Terrazas'**

In [58]:
df_terrazas = pd.read_csv(path_terrazas,sep = ";",skip_blank_lines = True,on_bad_lines = 'skip',engine = 'python',  encoding = 'utf-8')

In [59]:
campos_licencias = [
    'id_local', 'id_distrito_local', 'desc_distrito_local', 'id_barrio_local', 'desc_barrio_local',
    'cod_barrio_local', 'id_seccion_censal_local', 'desc_seccion_censal_local', 'coordenada_x_local',
    'coordenada_y_local', 'id_tipo_acceso_local', 'desc_tipo_acceso_local', 'id_situacion_local',
    'desc_situacion_local', 'id_vial_edificio', 'clase_vial_edificio', 'desc_vial_edificio',
    'id_ndp_edificio', 'id_clase_ndp_edificio', 'nom_edificio', 'num_edificio', 'cal_edificio',
    'secuencial_local_PC', 'id_vial_acceso', 'clase_vial_acceso', 'desc_vial_acceso',
    'id_ndp_acceso', 'id_clase_ndp_acceso', 'nom_acceso', 'num_acceso', 'cal_acceso',
    'coordenada_x_agrupacion', 'coordenada_y_agrupacion', 'id_agrupacion', 'nombre_agrupacion',
    'id_tipo_agrup', 'desc_tipo_agrup', 'id_planta_agrupado', 'id_local_agrupado', 'rotulo',
    'id_seccion', 'desc_seccion', 'id_division', 'desc_division', 'id_epigrafe', 'desc_epigrafe',
    'fx_carga', 'fx_datos_ini', 'fx_datos_fin'
]

campos_locales = df_locales.columns.tolist()

campos_exclusivos_licencias = [col for col in campos_licencias if col not in campos_locales]
print(campos_exclusivos_licencias)


['id_seccion', 'desc_seccion', 'id_division', 'desc_division', 'id_epigrafe', 'desc_epigrafe']


In [60]:
campos_locales = df_locales.columns.tolist()
print(campos_locales)

['id_local', 'id_distrito_local', 'desc_distrito_local', 'id_barrio_local', 'desc_barrio_local', 'cod_barrio_local', 'id_seccion_censal_local', 'desc_seccion_censal_local', 'coordenada_x_local', 'coordenada_y_local', 'id_tipo_acceso_local', 'desc_tipo_acceso_local', 'id_situacion_local', 'desc_situacion_local', 'id_vial_edificio', 'clase_vial_edificio', 'desc_vial_edificio', 'id_ndp_edificio', 'id_clase_ndp_edificio', 'nom_edificio', 'num_edificio', 'cal_edificio', 'secuencial_local_PC', 'id_vial_acceso', 'clase_vial_acceso', 'desc_vial_acceso', 'id_ndp_acceso', 'id_clase_ndp_acceso', 'nom_acceso', 'num_acceso', 'cal_acceso', 'coordenada_x_agrupacion', 'coordenada_y_agrupacion', 'id_agrupacion', 'nombre_agrupacion', 'id_tipo_agrup', 'desc_tipo_agrup', 'id_planta_agrupado', 'id_local_agrupado', 'rotulo', 'cod_postal', 'hora_apertura1', 'hora_apertura2', 'hora_cierre1', 'hora_cierre2', 'fx_carga', 'fx_datos_ini', 'fx_datos_fin']


## 1.1 CAMPOS DUPLICADOS

**Locales vs Actividades**

In [61]:
#Query por 'id_local'
resultado = df_locales.query("id_local == 20000756")
resultado.id_local,resultado.coordenada_y_local,resultado.id_tipo_acceso_local,resultado.id_tipo_acceso_local

(6    20000756
 Name: id_local, dtype: int64,
 6    4472755.5
 Name: coordenada_y_local, dtype: float64,
 6    1
 Name: id_tipo_acceso_local, dtype: int64,
 6    1
 Name: id_tipo_acceso_local, dtype: int64)

In [62]:
#Query por 'id_local'
resultado = df_actividad.query("id_local == 20000756")
resultado.id_local,resultado.coordenada_y_local,resultado.id_tipo_acceso_local,resultado.id_tipo_acceso_local

(6515    20000756
 Name: id_local, dtype: int64,
 6515    4472755.5
 Name: coordenada_y_local, dtype: float64,
 6515    1
 Name: id_tipo_acceso_local, dtype: int64,
 6515    1
 Name: id_tipo_acceso_local, dtype: int64)

In [63]:
#Query por 'id_local'
resultado = df_terrazas.query("id_local == 20000756")
resultado.id_local,resultado.coordenada_y_local,resultado.id_tipo_acceso_local,resultado.id_tipo_acceso_local

(5471    20000756
 Name: id_local, dtype: int64,
 5471    4472755.5
 Name: coordenada_y_local, dtype: float64,
 5471    1
 Name: id_tipo_acceso_local, dtype: int64,
 5471    1
 Name: id_tipo_acceso_local, dtype: int64)

**Como se puede comprobar hay campos que tienen los datos repetidos en ambos archivos por lo que la información descrita en los PDFs es veraz**
Lo que plantearé será dejar el dataframe "locales" como el archivo maestro y luego eliminaré de los demás archivos las dimensiones que posean datos repetidos, así solo quedarán los campos con información limpia y quedarán las futuras colecciones mejor estructuradas.

En el caso en que se quieran realizar consultas de union de tablas el ID **id_local** servirá con clave foranea en las demás tablas.

**Eliminacion de columnas con información duplicadas**

In [64]:

# 1. DataFrame de Locales (fuente principal)
# No hacemos cambios en df_locales

# 2. DataFrame de Actividades
# Mantener solo campos únicos de actividades
columnas_actividades = [
    'id_local','id_seccion',  # clave de unión
    'desc_seccion', 'id_division', 'desc_division',
    'id_epigrafe', 'desc_epigrafe'
]
df_actividad = df_actividad[columnas_actividades]

# 3. DataFrame de Licencias
# Mantener solo campos únicos de licencias (pero no aparecen en el índice proporcionado)
# Asumimos que deberían estar estas columnas basadas en los PDFs:
columnas_licencias = [
    'id_local',
    'ref_licencia',
    'id_tipo_licencia',
    'desc_tipo_licencia',
    'id_tipo_situacion_licencia',
    'desc_tipo_situacion_licencia',
    'Fecha_Dec_Lic']

# Filtramos solo las columnas que existan realmente
df_licencias = df_licencias[columnas_licencias]

# 4. DataFrame de Terrazas
# Mantener solo campos únicos de terrazas
columnas_terrazas= [
    "id_local",
    "id_terraza",
    "Escalera",
    "id_periodo_terraza",
    "desc_periodo_terraza",
    "id_situacion_terraza",
    "desc_situacion_terraza",
    "Superficie_ES",
    "Superficie_RA",
    "Fecha_confir_ult_decreto_resol",
    "id_ndp_terraza",
    "id_clase_ndp_terraza",
    "id_vial",
    "desc_clase",
    "desc_nombre",
    "nom_terraza",
    "num_terraza",
    "cal_terraza",
    "desc_ubicacion_terraza",
    "hora_ini_LJ_es",
    "hora_fin_LJ_es",
    "hora_ini_LJ_ra",
    "hora_fin_LJ_ra",
    "hora_ini_VS_es",
    "hora_fin_VS_es",
    "hora_ini_VS_ra",
    "hora_fin_VS_ra",
    "mesas_aux_es",
    "mesas_aux_ra",
    "mesas_es",
    "mesas_ra",
    "sillas_es",
    "sillas_ra",
    "fx_carga",
    "fx_datos_ini",
    "fx_datos_fin"
]

df_terrazas = df_terrazas[columnas_terrazas]


## 1.2 Limpieza y análisis de los DF

### 1.2.1 DF_LOCALES

In [65]:
df_locales.describe(include='all')

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,...,id_local_agrupado,rotulo,cod_postal,hora_apertura1,hora_apertura2,hora_cierre1,hora_cierre2,fx_carga,fx_datos_ini,fx_datos_fin
count,1.511620e+05,151162.000000,151162,151162.000000,151162,151162.000000,151162.000000,151162.000000,151162.000000,1.511620e+05,...,90445,151162,151162.000000,8796,812,6311,862,151162,151162,151162
unique,NaN,NaN,21,NaN,131,NaN,NaN,NaN,NaN,NaN,...,3467,80142,NaN,63,37,56,34,112,1,1
top,NaN,NaN,CENTRO,NaN,SAN ANDRES,NaN,NaN,NaN,NaN,NaN,...,01,SIN ACTIVIDAD,NaN,08:00,10:00,02:00,02:00,2023-12-08 07:01:00.937,2023-12-01,2023-12-01
freq,NaN,NaN,13440,NaN,4092,NaN,NaN,NaN,NaN,NaN,...,20570,39418,NaN,1977,435,3063,461,2434,151162,151162
mean,2.655094e+08,9.939661,NaN,997.419001,NaN,3.452925,10011.069158,71.408396,399836.581200,4.049711e+06,...,NaN,NaN,28025.110689,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,5.016712e+07,5.625185,NaN,562.402277,NaN,1.862671,5626.998295,48.532575,129534.909558,1.311569e+06,...,NaN,NaN,14.277943,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000e+07,1.000000,NaN,101.000000,NaN,1.000000,1001.000000,1.000000,0.000000,0.000000e+00,...,NaN,NaN,28001.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2.704840e+08,5.000000,NaN,505.000000,NaN,2.000000,5085.000000,32.000000,439092.600000,4.470752e+06,...,NaN,NaN,28014.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2.800378e+08,10.000000,NaN,1004.000000,NaN,3.000000,10137.000000,65.000000,440758.600000,4.473638e+06,...,NaN,NaN,28025.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2.850114e+08,15.000000,NaN,1501.000000,NaN,5.000000,15041.000000,104.000000,443691.620000,4.476700e+06,...,NaN,NaN,28037.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
#Revisión general del data frame
print(df_locales.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151162 entries, 0 to 151161
Data columns (total 48 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id_local                   151162 non-null  int64  
 1   id_distrito_local          151162 non-null  int64  
 2   desc_distrito_local        151162 non-null  object 
 3   id_barrio_local            151162 non-null  int64  
 4   desc_barrio_local          151162 non-null  object 
 5   cod_barrio_local           151162 non-null  int64  
 6   id_seccion_censal_local    151162 non-null  int64  
 7   desc_seccion_censal_local  151162 non-null  int64  
 8   coordenada_x_local         151162 non-null  float64
 9   coordenada_y_local         151162 non-null  float64
 10  id_tipo_acceso_local       151162 non-null  int64  
 11  desc_tipo_acceso_local     151162 non-null  object 
 12  id_situacion_local         151162 non-null  int64  
 13  desc_situacion_local       15

**Es recomendable trabajar con Object en vez de Strings así que todas las columnas tienen formatos alineados con su tipo de contenido o de dato**

In [67]:
print(df_locales.isnull().sum())

id_local                          0
id_distrito_local                 0
desc_distrito_local               0
id_barrio_local                   0
desc_barrio_local                 0
cod_barrio_local                  0
id_seccion_censal_local           0
desc_seccion_censal_local         0
coordenada_x_local                0
coordenada_y_local                0
id_tipo_acceso_local              0
desc_tipo_acceso_local            0
id_situacion_local                0
desc_situacion_local              0
id_vial_edificio                  0
clase_vial_edificio               0
desc_vial_edificio                0
id_ndp_edificio                   0
id_clase_ndp_edificio             0
nom_edificio                      0
num_edificio                      0
cal_edificio                      0
secuencial_local_PC               0
id_vial_acceso                    0
clase_vial_acceso                 0
desc_vial_acceso                  0
id_ndp_acceso                     0
id_clase_ndp_acceso         

#  Informe: Justificación de Valores Nulos en el Dataset `locales.csv`

Este informe detalla las razones por las cuales la presencia de valores nulos en determinados campos del archivo `locales.csv` del Censo de Locales y Actividades del Ayuntamiento de Madrid es **esperable, coherente** y **estructuralmente justificada**.

La documentación técnica consultada incluye:
-  *Estructura_DS_FicheroCLA.pdf*
-  *Estructura_DS_FicheroCLA_LICENCIAS_cp.pdf*
-  *Estructura_DS_FicheroCLA_Terrazas_cp.pdf*

---

##  1. Campos de Agrupación

Los campos relacionados con agrupaciones de locales (como centros comerciales, mercados, etc.) **solo aplican** a locales con tipo de acceso **"Agrupado"** (`id_tipo_acceso_local = 0`).

| Campo | Nulos | Justificación |
|-------|-------|---------------|
| `coordenada_x_agrupacion`<br>`coordenada_y_agrupacion` | 136.661 | Solo se completan para locales agrupados. Los locales tipo "Puerta de Calle" o "Asociado" no deben tener estas coordenadas. |
| `id_planta_agrupado` | 1.648 | Aplica solo a locales dentro de agrupaciones. |
| `id_local_agrupado` | 60.717 | Solo se utiliza para identificar locales dentro de agrupaciones. |
| `id_agrupacion`, `nombre_agrupacion`, `id_tipo_agrup`, `desc_tipo_agrup` | 0 (sin nulos) | Estos campos están presentes siempre, pero para locales no agrupados su valor será `"SIN AGRUPACION"` o `-1`, lo cual es correcto. |

 **Conclusión**: La mayoría de locales no están agrupados, por lo que la ausencia de valores en estos campos es completamente normal y coherente con la estructura del censo.

---

##  2. Campos de Horarios

Los campos de apertura (`hora_apertura1`, `hora_apertura2`) están **condicionados por el tipo de actividad del local**.

| Campo | Nulos | Justificación |
|-------|-------|---------------|
| `hora_apertura1` | 142.366 | Muchos locales (oficinas, almacenes, despachos) no tienen obligación de registrar horarios. |
| `hora_apertura2` | 150.350 | No todos los locales operan en doble jornada o turno partido. |

 **Conclusión**: Estos campos dependen del tipo de uso del local y no son aplicables en todos los casos.

---

##  3. Campo `cal_acceso`

| Campo | Nulos | Justificación |
|-------|-------|---------------|
| `cal_acceso` | 0 | Aunque no hay nulos, el contenido puede estar vacío cuando el número de acceso no requiere calificador (letras como A, B, DUP, etc.). Es opcional según la dirección. |

 **Conclusión**: La omisión de este dato no implica un error, sino la ausencia de un calificador aplicable.

---

##  Recomendación General

Para cualquier análisis, se debe tener en cuenta que:
- Los **valores nulos responden a la lógica del tipo de local**, su acceso, su agrupación y si tiene o no usos especiales (como terrazas o licencias).
- **No deben imputarse ni eliminarse automáticamente**, sino gestionarse según el contexto semántico de cada variable.

---


In [68]:
#Comprobación de la calidad de los Nulos.
print(df_locales['id_tipo_acceso_local'].value_counts())

id_tipo_acceso_local
1     134010
0      14501
12      2651
Name: count, dtype: int64


In [69]:
agrupados = df_locales[df_locales['id_tipo_acceso_local'] == 0]
print(agrupados[['coordenada_x_agrupacion', 'id_planta_agrupado']].isnull().sum())

coordenada_x_agrupacion      0
id_planta_agrupado         989
dtype: int64


In [70]:
puerta_calle = df_locales[df_locales['id_tipo_acceso_local'] == 1]
print(puerta_calle[['coordenada_x_agrupacion', 'id_planta_agrupado']].isnull().sum())

coordenada_x_agrupacion    134010
id_planta_agrupado             12
dtype: int64


**Se va a poner un valor meramente orientativo para rellenar los campos que contienen valores Nulos, con el fin de que a futuro el consumidor obtenga los datos depurados y listos para el consumo**

In [71]:
# Coordenadas de agrupación (float)
df_locales['coordenada_x_agrupacion'] = df_locales['coordenada_x_agrupacion'].fillna(0.0)
df_locales['coordenada_y_agrupacion'] = df_locales['coordenada_y_agrupacion'].fillna(0.0)

# Plantas y números de local (int)
df_locales['id_planta_agrupado'] = df_locales['id_planta_agrupado'].fillna(-99)  # Código especial
df_locales['id_local_agrupado'] = df_locales['id_local_agrupado'].fillna(-99)

# Horarios (asumiendo formato "hh:mm:ss")
df_locales['hora_apertura1'] = df_locales['hora_apertura1'].fillna('00:00:00')
df_locales['hora_apertura2'] = df_locales['hora_apertura2'].fillna('00:00:00')

# Relleno de horarios de cierre (asumiendo formato string "HH:MM:SS")
df_locales['hora_cierre1'] = df_locales['hora_cierre1'].fillna('00:00:00')
df_locales['hora_cierre2'] = df_locales['hora_cierre2'].fillna('00:00:00')

In [72]:
#COMPROBACIÓN FINAL DE NULOS
print(df_locales.isnull().sum())

id_local                     0
id_distrito_local            0
desc_distrito_local          0
id_barrio_local              0
desc_barrio_local            0
cod_barrio_local             0
id_seccion_censal_local      0
desc_seccion_censal_local    0
coordenada_x_local           0
coordenada_y_local           0
id_tipo_acceso_local         0
desc_tipo_acceso_local       0
id_situacion_local           0
desc_situacion_local         0
id_vial_edificio             0
clase_vial_edificio          0
desc_vial_edificio           0
id_ndp_edificio              0
id_clase_ndp_edificio        0
nom_edificio                 0
num_edificio                 0
cal_edificio                 0
secuencial_local_PC          0
id_vial_acceso               0
clase_vial_acceso            0
desc_vial_acceso             0
id_ndp_acceso                0
id_clase_ndp_acceso          0
nom_acceso                   0
num_acceso                   0
cal_acceso                   0
coordenada_x_agrupacion      0
coordena

# 1.2.2 DF_Actividad

In [73]:
df_actividad.describe(include='all')

,id_local,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe
count,1.695590e+05,169559,169559,169559,169559,169559,169559
unique,NaN,22,22,83,83,439,439
top,NaN,G,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,47,"COMERCIO AL POR MENOR, EXCEPTO DE VEHÍCULOS DE...",-1,VALOR NULO EN ORIGEN
freq,NaN,51516,51516,44670,44670,39292,39292
mean,2.632886e+08,NaN,NaN,NaN,NaN,NaN,NaN
std,5.323945e+07,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000e+07,NaN,NaN,NaN,NaN,NaN,NaN
25%,2.704429e+08,NaN,NaN,NaN,NaN,NaN,NaN
50%,2.800304e+08,NaN,NaN,NaN,NaN,NaN,NaN
75%,2.850085e+08,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
print(df_actividad.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169559 entries, 0 to 169558
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id_local       169559 non-null  int64 
 1   id_seccion     169559 non-null  object
 2   desc_seccion   169559 non-null  object
 3   id_division    169559 non-null  object
 4   desc_division  169559 non-null  object
 5   id_epigrafe    169559 non-null  object
 6   desc_epigrafe  169559 non-null  object
dtypes: int64(1), object(6)
memory usage: 9.1+ MB
None


In [75]:
print(df_actividad.isnull().sum())

id_local         0
id_seccion       0
desc_seccion     0
id_division      0
desc_division    0
id_epigrafe      0
desc_epigrafe    0
dtype: int64


**Cada columna tiene su valor y su tipo de dato alineado con un buen trato del dato.**

## 1.2.3 DF_licencias

In [76]:
df_licencias.head()

,id_local,ref_licencia,id_tipo_licencia,desc_tipo_licencia,id_tipo_situacion_licencia,desc_tipo_situacion_licencia,Fecha_Dec_Lic
0,270366725,350/2023/26343,4,Transmisión de licencia Urbanística,5,Transmisión de Licencia Concedida,01/01/1900
1,270433440,108/2007/07166,2,Licencia Urbanística,2,Concedida,31/08/2009
2,270433462,500/2015/05608,1,Declaración Responsable,1,En tramitación,10/04/2015
3,270433466,220/2011/13207,4,Transmisión de licencia Urbanística,1,En tramitación,15/11/2011
4,270433476,107/2004/07541,2,Licencia Urbanística,2,Concedida,14/06/2005


In [77]:
df_licencias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150829 entries, 0 to 150828
Data columns (total 7 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   id_local                      150829 non-null  int64 
 1   ref_licencia                  150829 non-null  object
 2   id_tipo_licencia              150829 non-null  int64 
 3   desc_tipo_licencia            150829 non-null  object
 4   id_tipo_situacion_licencia    150829 non-null  int64 
 5   desc_tipo_situacion_licencia  150829 non-null  object
 6   Fecha_Dec_Lic                 150829 non-null  object
dtypes: int64(3), object(4)
memory usage: 8.1+ MB


In [78]:
df_licencias.isnull().sum()

id_local                        0
ref_licencia                    0
id_tipo_licencia                0
desc_tipo_licencia              0
id_tipo_situacion_licencia      0
desc_tipo_situacion_licencia    0
Fecha_Dec_Lic                   0
dtype: int64

## 1.2.4 DF_TERRAZAS

In [79]:
df_terrazas.head()

,id_local,id_terraza,Escalera,id_periodo_terraza,desc_periodo_terraza,id_situacion_terraza,desc_situacion_terraza,Superficie_ES,Superficie_RA,Fecha_confir_ult_decreto_resol,...,hora_fin_VS_ra,mesas_aux_es,mesas_aux_ra,mesas_es,mesas_ra,sillas_es,sillas_ra,fx_carga,fx_datos_ini,fx_datos_fin
0,270403150,33,NaN,1,Anual,1,Abierta,21.06,16.20,04/04/2014,...,00:00:00,0,0.0,9,9.0,27,27.0,2023-12-08 07:00:49.223,2023-12-01,2023-12-01
1,80000291,1168,NaN,1,Anual,1,Abierta,89.04,89.04,15/12/2014,...,00:00:00,0,0.0,22,22.0,70,70.0,2023-12-08 07:00:49.227,2023-12-01,2023-12-01
2,40002970,42,NaN,1,Anual,1,Abierta,14.89,14.89,07/04/2014,...,00:00:00,0,0.0,4,0.0,16,0.0,2023-12-08 07:00:49.223,2023-12-01,2023-12-01
3,40002488,1176,NaN,1,Anual,1,Abierta,27.00,27.00,17/12/2014,...,00:00:00,0,0.0,18,18.0,36,36.0,2023-12-08 07:00:49.227,2023-12-01,2023-12-01
4,170000773,56,NaN,2,Estacional,1,Abierta,12.96,NaN,14/04/2014,...,NaN,0,0.0,4,0.0,16,0.0,2023-12-08 07:00:49.223,2023-12-01,2023-12-01


In [80]:
df_terrazas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6788 entries, 0 to 6787
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id_local                        6788 non-null   int64  
 1   id_terraza                      6788 non-null   int64  
 2   Escalera                        123 non-null    object 
 3   id_periodo_terraza              6788 non-null   int64  
 4   desc_periodo_terraza            6788 non-null   object 
 5   id_situacion_terraza            6788 non-null   int64  
 6   desc_situacion_terraza          6788 non-null   object 
 7   Superficie_ES                   6788 non-null   float64
 8   Superficie_RA                   5610 non-null   float64
 9   Fecha_confir_ult_decreto_resol  6788 non-null   object 
 10  id_ndp_terraza                  6788 non-null   int64  
 11  id_clase_ndp_terraza            6788 non-null   int64  
 12  id_vial                         67

In [81]:
df_terrazas.isnull().sum()

id_local                             0
id_terraza                           0
Escalera                          6665
id_periodo_terraza                   0
desc_periodo_terraza                 0
id_situacion_terraza                 0
desc_situacion_terraza               0
Superficie_ES                        0
Superficie_RA                     1178
Fecha_confir_ult_decreto_resol       0
id_ndp_terraza                       0
id_clase_ndp_terraza                 0
id_vial                              0
desc_clase                           0
desc_nombre                          0
nom_terraza                          0
num_terraza                          0
cal_terraza                          0
desc_ubicacion_terraza               0
hora_ini_LJ_es                       0
hora_fin_LJ_es                       0
hora_ini_LJ_ra                    1178
hora_fin_LJ_ra                    1178
hora_ini_VS_es                       0
hora_fin_VS_es                       0
hora_ini_VS_ra           

**En los demas ficheros CSV los titulos estan normalizados en minuscula pero terrazas tiene algunos campos con mayusculas y minusculas de por medio, por lo que voy a normalizarlos para que en futuras consultas sea mas comodo y se eviten errores**

In [82]:
df_terrazas.columns = df_terrazas.columns.str.lower().str.replace(' ', '_')

In [83]:
# Diccionario con valores por defecto para las columnas que tienen NaN
fill_dict = {
    'Escalera':"-99",
    'superficie_ra': 0.0,      # Rellenamos la superficie en el resto del año con 0.0
    'mesas_aux_ra': 0,         # Si no se especifica mesas auxiliares en RA, asumimos 0
    'mesas_ra': 0.0,           # Rellenamos con 0.0 mesas en RA
    'sillas_ra': 0.0,          # Rellenamos con 0.0 sillas en RA
    'hora_ini_lj_ra': "00:00:00",  # Valor por defecto en formato de hora para el inicio en RA en días L-J
    'hora_fin_lj_ra': "00:00:00",  # Valor por defecto para el fin en RA en días L-J
    'hora_ini_vs_ra': "00:00:00",  # Valor por defecto para el inicio en RA en VS
    'hora_fin_vs_ra': "00:00:00"   # Valor por defecto para el fin en RA en VS
}

# Aplicar fillna con los valores indicados
df_terrazas.fillna(value=fill_dict, inplace=True)


In [84]:
# Verificamos que ya no queden valores nulos en esas columnas:
print(df_terrazas[['superficie_ra', 'mesas_aux_ra', 'mesas_ra', 'sillas_ra',
                    'hora_ini_lj_ra', 'hora_fin_lj_ra', 'hora_ini_vs_ra', 'hora_fin_vs_ra']].isnull().sum())

superficie_ra     0
mesas_aux_ra      0
mesas_ra          0
sillas_ra         0
hora_ini_lj_ra    0
hora_fin_lj_ra    0
hora_ini_vs_ra    0
hora_fin_vs_ra    0
dtype: int64


## 1.3 Depuración Final
En este punto se van a tomar los data frames  y se van a eliminar los espacios en blancos en los campos object para que todo el texto quede ordenador. Y los espacios intermedios se van a sustituir por '_'.


In [85]:
def limpiar_df(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    df = df.map(lambda x: x.strip().lower() if isinstance(x, str) else x)
    #df = df.drop_duplicates() -- Si a futuro hiciera falta elimminar duplicados de algun data frame.
    return df

df_locales = limpiar_df(df_locales)
df_actividad = limpiar_df(df_actividad)
df_licencias = limpiar_df(df_licencias)
df_terrazas = limpiar_df(df_terrazas)

In [86]:
df_terrazas.head()

,id_local,id_terraza,escalera,id_periodo_terraza,desc_periodo_terraza,id_situacion_terraza,desc_situacion_terraza,superficie_es,superficie_ra,fecha_confir_ult_decreto_resol,...,hora_fin_vs_ra,mesas_aux_es,mesas_aux_ra,mesas_es,mesas_ra,sillas_es,sillas_ra,fx_carga,fx_datos_ini,fx_datos_fin
0,270403150,33,NaN,1,anual,1,abierta,21.06,16.20,04/04/2014,...,00:00:00,0,0.0,9,9.0,27,27.0,2023-12-08 07:00:49.223,2023-12-01,2023-12-01
1,80000291,1168,NaN,1,anual,1,abierta,89.04,89.04,15/12/2014,...,00:00:00,0,0.0,22,22.0,70,70.0,2023-12-08 07:00:49.227,2023-12-01,2023-12-01
2,40002970,42,NaN,1,anual,1,abierta,14.89,14.89,07/04/2014,...,00:00:00,0,0.0,4,0.0,16,0.0,2023-12-08 07:00:49.223,2023-12-01,2023-12-01
3,40002488,1176,NaN,1,anual,1,abierta,27.00,27.00,17/12/2014,...,00:00:00,0,0.0,18,18.0,36,36.0,2023-12-08 07:00:49.227,2023-12-01,2023-12-01
4,170000773,56,NaN,2,estacional,1,abierta,12.96,0.00,14/04/2014,...,00:00:00,0,0.0,4,0.0,16,0.0,2023-12-08 07:00:49.223,2023-12-01,2023-12-01


In [87]:
df_licencias.head()

,id_local,ref_licencia,id_tipo_licencia,desc_tipo_licencia,id_tipo_situacion_licencia,desc_tipo_situacion_licencia,fecha_dec_lic
0,270366725,350/2023/26343,4,transmisión de licencia urbanística,5,transmisión de licencia concedida,01/01/1900
1,270433440,108/2007/07166,2,licencia urbanística,2,concedida,31/08/2009
2,270433462,500/2015/05608,1,declaración responsable,1,en tramitación,10/04/2015
3,270433466,220/2011/13207,4,transmisión de licencia urbanística,1,en tramitación,15/11/2011
4,270433476,107/2004/07541,2,licencia urbanística,2,concedida,14/06/2005


In [88]:
df_actividad.head()

,id_local,id_seccion,desc_seccion,id_division,desc_division,id_epigrafe,desc_epigrafe
0,270574777,p,educación,85,educación,855002,"enseñanza no reglada (deportiva y recreativa, ..."
1,270574779,p,educación,85,educación,855002,"enseñanza no reglada (deportiva y recreativa, ..."
2,270574786,p,educación,85,educación,855002,"enseñanza no reglada (deportiva y recreativa, ..."
3,270574793,g,comercio al por mayor y al por menor; reparaci...,47,"comercio al por menor, excepto de vehículos de...",472206,"comercio al por menor de aves, huevos y caza s..."
4,270574793,g,comercio al por mayor y al por menor; reparaci...,47,"comercio al por menor, excepto de vehículos de...",472401,comercio al por menor de pan y productos de pa...


In [89]:
df_locales.head()

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,...,id_local_agrupado,rotulo,cod_postal,hora_apertura1,hora_apertura2,hora_cierre1,hora_cierre2,fx_carga,fx_datos_ini,fx_datos_fin
0,20000596,2,arganzuela,203,chopera,3,2043,43,440788.6,4471857.5,...,d1,la caña,28045,09:00,00:00:00,00:00:00,22:00,2023-12-08 07:01:00.047,2023-12-01,2023-12-01
1,20000605,2,arganzuela,201,imperial,1,2107,107,439316.6,4472540.5,...,01,cafeteria piramides,28005,00:00:00,00:00:00,00:00:00,00:00:00,2023-12-08 07:01:00.047,2023-12-01,2023-12-01
2,20000669,2,arganzuela,202,acacias,2,2096,96,439770.6,4472407.5,...,m1,bar melilla,28005,00:00:00,00:00:00,00:00:00,00:00:00,2023-12-08 07:01:00.047,2023-12-01,2023-12-01
3,20000709,2,arganzuela,202,acacias,2,2029,29,440355.6,4472282.5,...,iz,supermercados simply,28005,00:00:00,00:00:00,00:00:00,00:00:00,2023-12-08 07:01:00.047,2023-12-01,2023-12-01
4,20000721,2,arganzuela,201,imperial,1,2089,89,439312.6,4472870.5,...,c,arroceria imperial,28005,00:00:00,00:00:00,00:00:00,00:00:00,2023-12-08 07:01:00.047,2023-12-01,2023-12-01


# 2. EVALUACION DE EMBEBER ALGUN ARCHIVO DENTRO DE OTRO

In [90]:
# Calcular cuántas actividades hay por local
actividades_por_local = df_actividad['id_local'].value_counts()
print(actividades_por_local.describe())

count    151161.000000
mean          1.121711
std           0.477066
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          14.000000
Name: count, dtype: float64


En promedio cada local tiene 1.12 actividades lo cual es una luz verde para poder embeber actividades dentro locales. Ya que todo estaria bajo una misma dimensión y no habría que hacer referencias por ID_LOCAL ya estarían en la colección.
Por otro lado vemos que hay un MAX de 14 actividades por local esto será una casuistica por lo que en mi opinión no veo ninguna problema en embeber esta información. Sólo tendria un nivel de bajada dentro del campo 'actividades' dentro de 'locales'

In [91]:
# Calcular cuántas  licencias hay por local
licencias_por_local = df_licencias['id_local'].value_counts()
print(licencias_por_local.describe())

count    74139.000000
mean         2.034408
std          1.495419
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max         51.000000
Name: count, dtype: float64


## Decisión de modelado: Licencias como colección separada

Después de analizar estadísticamente la distribución de licencias por local (`id_local`), se ha determinado que no es eficiente ni escalable embeber las licencias dentro del documento principal de `locales`.

### Métricas clave:

- **Locales con licencias (`count`)**: 74,139
- **Promedio de licencias por local (`mean`)**: 2.03
- **Mediana (`50%`)**: 2 licencias por local
- **Máximo (`max`)**: 51 licencias para un único local

### Problemas al embeber licencias:

- **Tamaño excesivo de documento**: Algunos locales contienen un número muy alto de licencias (hasta 51), lo cual puede generar documentos pesados y de difícil mantenimiento.
- **Acceso independiente frecuente**: Las licencias son entidades que suelen consultarse de forma autónoma por fecha, tipo o estado (`ACTIVA`, `CADUCADA`, etc.), lo cual justifica su separación.
- **Escalabilidad**: Al crecer el sistema o añadir históricos de licencias, embebido dificultaría actualizaciones y escalabilidad horizontal.

Este enfoque se ajusta mejor al principio de **modelado normalizado** cuando hay relaciones de uno-a-muchos con alta cardinalidad.



## 2.1 EMBEBER ARCHIVO ACTIVIDAD 

In [ ]:
# Agrupar actividades por id_local
actividades_grouped = df_actividad.groupby('id_local').apply(lambda x: x.drop(columns='id_local').to_dict(orient='records'))

In [93]:
#Union en el data frame LOCALES
df_locales = df_locales.set_index('id_local')

df_locales['actividades'] = df_locales.index.map(actividades_grouped)

df_locales = df_locales.reset_index()


In [42]:
df_locales.head()

,id_local,id_distrito_local,desc_distrito_local,id_barrio_local,desc_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,...,rotulo,cod_postal,hora_apertura1,hora_apertura2,hora_cierre1,hora_cierre2,fx_carga,fx_datos_ini,fx_datos_fin,actividades
0,20000596,2,arganzuela,203,chopera,3,2043,43,440788.6,4471857.5,...,la caña,28045,09:00,00:00:00,00:00:00,22:00,2023-12-08 07:01:00.047,2023-12-01,2023-12-01,[{'desc_seccion': 'comercio al por mayor y al ...
1,20000605,2,arganzuela,201,imperial,1,2107,107,439316.6,4472540.5,...,cafeteria piramides,28005,00:00:00,00:00:00,00:00:00,00:00:00,2023-12-08 07:01:00.047,2023-12-01,2023-12-01,"[{'desc_seccion': 'hostelería', 'id_division':..."
2,20000669,2,arganzuela,202,acacias,2,2096,96,439770.6,4472407.5,...,bar melilla,28005,00:00:00,00:00:00,00:00:00,00:00:00,2023-12-08 07:01:00.047,2023-12-01,2023-12-01,"[{'desc_seccion': 'hostelería', 'id_division':..."
3,20000709,2,arganzuela,202,acacias,2,2029,29,440355.6,4472282.5,...,supermercados simply,28005,00:00:00,00:00:00,00:00:00,00:00:00,2023-12-08 07:01:00.047,2023-12-01,2023-12-01,[{'desc_seccion': 'comercio al por mayor y al ...
4,20000721,2,arganzuela,201,imperial,1,2089,89,439312.6,4472870.5,...,arroceria imperial,28005,00:00:00,00:00:00,00:00:00,00:00:00,2023-12-08 07:01:00.047,2023-12-01,2023-12-01,"[{'desc_seccion': 'hostelería', 'id_division':..."


## Conclusión del modelado NoSQL

Tras el análisis de cardinalidad, redundancia y uso previsto de los datos, se ha definido la siguiente estructura final para el modelo documental en MongoDB:

### Colección `locales`
Contendrá los datos principales de cada local (dirección, coordenadas, rotulo, etc.) y una lista **embebida** de sus actividades económicas:

- Modelo: `1 local ⟶ N actividades`
- Justificación: la mayoría de locales tiene solo una actividad (`mean = 1.12`), con un máximo de 14.
- Ventaja: acceso rápido y cohesivo desde un único documento.

---

### Colección `licencias`
Cada documento representa una licencia individual y contiene una referencia al `id_local`:

- Modelo: `1 local ⟶ muchas licencias`
- Justificación: algunos locales tienen hasta 51 licencias; la media es 2.03.
- Motivo de colección separada: alta cardinalidad y necesidad de consultas independientes (por tipo, estado, fecha...).

---

###  Colección `terrazas`
Al igual que `licencias`, cada terraza será un documento independiente asociado a un `id_local`:

- Modelo: `1 local ⟶ varias terrazas`
- Justificación: contiene muchos campos específicos que no deben incluirse en `locales`.
- Ventaja: evita documentos grandes o anidados innecesariamente.

---

> Con este diseño híbrido (embebido + referenciado), se logra un equilibrio entre rendimiento, escalabilidad y facilidad de mantenimiento en el modelo NoSQL.


# 3 Ingesta de datos a MongoDB

**Codigo para exportar desde el notebook a la base de datos nosql**

In [17]:
from pymongo import MongoClient

In [52]:
'''
#Conectarse a MongoDB local
client = MongoClient('localhost', 27017)
#Seleccionar base de datos
db = client["ucm_nosql_pl"]

# Insertar df_locales con actividades embebidas
variable_locales = df_locales.to_dict(orient='records')
db.locales.drop() #borrar la coleccion si ya existe y se quiere hacer una nueva
db.locales.insert_many(variable_locales)

# Insertar licencias
licencias_docs = df_licencias.to_dict(orient="records")
#db.licencias.drop()
db.licencias.insert_many(licencias_docs)

# Insertar terrazas
terrazas_docs = df_terrazas.to_dict(orient="records")
db.terrazas.drop()
db.terrazas.insert_many(terrazas_docs)

print("Datos cargados correctamente en MongoDB")
'''

Datos cargados correctamente en MongoDB


![MongoDB_Carga_Exitosa](imagenes_capturas_mongo/MongoDB.png)

## OBTENER LOS DATA FRAME EN CSV PARA PASARLOS A NEO4J

In [94]:
'''
# Exportar DataFrames a CSV para carga en Neo4j

# 1. Locales
df_locales.to_csv("locales.csv", index=False)

# 2. Licencias
df_licencias.to_csv("licencias.csv", index=False)

# 3. Terrazas
df_terrazas.to_csv("terrazas.csv", index=False)
'''
